# Project 3 - Water Pump Clasificaiton

# Setup

In [1]:
from __future__ import print_function

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from __future__ import division
pd.set_option('display.width',5000)

In [3]:
import patsy

from sklearn import linear_model as lm
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation
from sklearn import metrics
from sklearn.metrics import confusion_matrix

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Data Import - Training Data

This data is part of the Data Driven Competition

https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/


In [19]:
# 'Values' provided for each pump location - features

df_values = pd.read_csv('/Users/amycurneen/ds/metis/metisgh/Metis-Curneen/3 - Water Pumps/Data Downloads/Training set values.csv')
df_values.sample(3)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
47375,34731,0.0,2011-04-07,NaN,0,NaN,33.874989,-9.306254,Ngana,0,...,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
45756,24371,0.0,2011-02-27,Private,33,private,39.301833,-6.975247,Kwa Mzee Njaule,0,...,never pay,soft,good,enough,enough,machine dbh,borehole,groundwater,other,other
58687,66562,0.0,2013-02-22,Kkkt,1321,KKT,38.004546,-4.299448,Nazaleti,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe


In [5]:
# 'Labels' provided for each pump location - clasificaiton - what I am predicting

df_labels = pd.read_csv('/Users/amycurneen/ds/metis/metisgh/Metis-Curneen/3 - Water Pumps/Data Downloads/Training set labels.csv')
df_labels.sample(5)

,id,status_group
10011,580,functional
49459,63098,non functional
3136,31599,non functional
22113,25805,non functional
46757,13474,non functional


# Exploratory Data Analysis

## Colum descriptions

My goal is to predict the operating condition of a waterpoint for each record in the dataset. I was provided the following set of information about the waterpoints:

* amount_tsh - Total static head (amount water available to waterpoint)
    * 98 unique
* date_recorded - The date the row was entered
    * 365 unique
    * year - month - day
* funder - Who funded the well
    * 1897 unique
    * look at top ones?
* installer - Organization that installed the well
    * 2145 unique
    * DWE is main one - 10x closest other, 17k
* wpt_name - Name of the waterpoint if there is one
    * 37400 unique
    * look at top ones?
* num_private - (NO PROVIDED DESC)
    * 65 unique
    * USELESS FEATURE
* population - Population around the well
    * 1049 unique
    * a lot are zero
* public_meeting - True/False
    * 2 unique
* recorded_by - Group entering this row of data
    * 1 unique
    * all the same - USELESS FEATURE
* scheme_management - Who operates the waterpoint
    * 12 unique
* scheme_name - Who operates the waterpoint
    * 2696 unique
    * USELESS FEATURE
* permit - If the waterpoint is permitted
    * 2 unique
* construction_year - Year the waterpoint was constructed
    * 55 unique
    * third are 0 - USELESS FEATURE


* Geography
    * gps_height - Altitude of the well
        * numerical
    * longitude - GPS coordinate
        * numerical
    * latitude - GPS coordinate
        * numerical
    * basin - Geographic water basin
        * 9 unique
    * subvillage - Geographic location
        * 19287 unique
    * region - Geographic location
        * 21 unique
    * region_code - Geographic location (coded)
        * 27 unique
    * district_code - Geographic location (coded)
        * 20 unique
    * lga - Geographic location
        * 125 unique
    * ward - Geographic location
        * 2092 unique


* Extraction
    * extraction_type - The kind of extraction the waterpoint uses
        * 18 unique
        * Most descriptive of extraction
    * extraction_type_group - The kind of extraction the waterpoint uses
        * 13 unique
        * Parent of extraction_type
    * extraction_type_class - The kind of extraction the waterpoint uses
        * 7 unique
        * Parent of extraction_type_group


* Overhead
    * management - How the waterpoint is managed
        * 12 unique
    * management_group - How the waterpoint is managed
        * 5 unique
    * payment - What the water costs
        * 7 unique
        * same as payment type
    * payment_type - What the water costs
        * 7 unique
        * same as payment


* Water
    * water_quality - The quality of the water 
        * 3 unique
        * Subset of quality_group
    * quality_group - The quality of the water
        * 6 unique
        * Parent group of water_quality
    * quantity - The quantity of water
        * 5 unique
        * Same as quantity_group
    * quantity_group - The quantity of water
        * 5 unique
        * Same as quantity
    * source - The source of the water
        * 10 unique
    * source_type - The source of the water
        * 7 unique
        * Subset of source
    * source_class - The source of the water
        * 3 unique
        * Subset of source_type
    * waterpoint_type - The kind of waterpoint
        * 6 unique
        * Parent of waterpoint_type_group
    * waterpoint_type_group - The kind of waterpoint
        * 7 unique
        * Subset of waterpoint_type

## Various EDA

##### Status

In [ ]:
# 3 Unique status present
df_labels.status_group.value_counts()

##### Finding features

##### Review histograms of extraction data

* extraction_type - The kind of extraction the waterpoint uses
* extraction_type_group - The kind of extraction the waterpoint uses
* extraction_type_class - The kind of extraction the waterpoint uses

In [ ]:
df_values.extraction_type.value_counts().plot('barh')
len(df_values.extraction_type.value_counts())

In [ ]:
df_values.extraction_type_group.value_counts().plot('barh')
len(df_values.extraction_type_group.value_counts())

In [ ]:
df_values.extraction_type_class.value_counts().plot('barh')
len(df_values.extraction_type_class.value_counts())

##### Other

In [ ]:
# df_values.population.value_counts()

In [ ]:
len(df_values.ward.value_counts())

In [ ]:
# len(df_values.management.value_counts())

In [ ]:
# print(df_values.region.value_counts())
# print('Length: ',len(df_values.region.value_counts()))

In [ ]:
#print(df_values.region_code.value_counts())
#print('Length: ',len(df_values.region_code.value_counts()))

In [ ]:
# df_values.water_quality.value_counts()

In [ ]:
#df_values.installer.value_counts()

# Test Train Split

In [6]:
# randomize data
# df_lables, df_values - combine and shuffle this data

df = pd.merge(df_labels,df_values,how = 'left')
df = df.sample(frac=1).reset_index(drop=True)

In [7]:
# Split back to X and y and then TTS
y = df[["id","status_group"]]
X = df.drop('status_group', axis=1)

from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Data Import - Challenge Data

In [15]:
# 'Values' provided for each competition pump location - features

df_test_values = pd.read_csv('/Users/amycurneen/ds/metis/metisgh/Metis-Curneen/3 - Water Pumps/Data Downloads/Test set values.csv')
df_test_values.sample(3)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
1004,30892,100.0,2013-02-05,NaN,1378,NaN,34.656241,-5.553928,Mkiwa Secondary,0,...,per bucket,soft,good,insufficient,insufficient,machine dbh,borehole,groundwater,communal standpipe,communal standpipe
12624,50924,50.0,2013-02-27,Wateraid,2097,ADRA,35.428741,-4.242044,Kwa Longini,0,...,per bucket,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe multiple,communal standpipe
3347,31662,0.0,2011-04-02,World Bank,0,Commu,35.615102,-6.289807,Kwa Edward,0,...,never pay,coloured,colored,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump


In [16]:
# 'Labels' I will provide for each pump location - clasificaiton

df_sub = pd.read_csv('/Users/amycurneen/ds/metis/metisgh/Metis-Curneen/3 - Water Pumps/Data Downloads/SubmissionFormat.csv')
df_sub.sample(3)

,id,status_group
490,31787,predicted label
4977,13433,predicted label
14167,27598,predicted label


In [17]:
df_sub = df_sub.drop('status_group', axis = 1)

# Dummy Classification

Functional is the highest output

## Dummy code

In [18]:
# set everything to functional - most common - and submit for a baseline

df_sub['status_group'] = pd.Series('functional', index=df_sub.index)
df_sub.to_csv('./Submissions/dummy_classification.csv',index=False)

## Result for submission

<table>
  <tr>
    <th>Best</th>
    <th>Current Rank</th>
    <th>Competitors</th>
    <th>Time</th>
  </tr>
  <tr>
    <td>0.5461</td>
    <td>1342</td>
    <td>5030</td>
    <td>May 3, 2018, 8:37 p.m.</td>
  </tr>
</table>